In [7]:
from pydantic import BaseModel
from utils import llm_service
import importlib
importlib.reload(llm_service)

<module 'utils.llm_service' from '/Users/hosseinb/projects/llm-orchestrated-computational-intelligence/src/utils/llm_service.py'>

In [3]:

resp = llm_service.get_chat_completion(llm_service.Model.GLM_4_7.value, 
[
    {"role": "system", "content": "Hello, world! what do you think?"}, 
    {"role": "user", "content": "what is 2+2?"}
], 
max_tokens=200)
resp

'2 + 2 is 4.'

In [8]:
class Answer(BaseModel):
    answer: int
    reason: str

resp = llm_service.get_chat_completion_structured(llm_service.Model.GLM_4_7.value, 
[
    {"role": "system", "content": "Hello, world! what do you think?"}, 
    {"role": "user", "content": "what is 2+2?"}
], 
Answer,
max_tokens=200)
resp

Answer(answer=4, reason='Basic arithmetic addition.')